In [1]:
from utils.processor import *
from utils.solver import *
from config import *

In [2]:
data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')
origin = (HOME['lat'], HOME['lon'])
dest = (PACIFIC_CENTER['lat'], PACIFIC_CENTER['lon'])
topK = 10
stay_duration = 1.5 * 60 
parking_budget = 5.00

processor = Processor(datapath, origin, dest, topK, stay_duration, parking_budget)
variables = processor.form_variables_dict()
variables

(7954, 21)
(618, 23)
IDX - 0
Time limit: 2 	 Rate: 5.0
*** Travel time: 17.383333333333333 ***
out: 1 - in: 0
~~~ Parking Status: 2 ~~~
{'b_1': 1.0, 'constraint_coeffs': {'parking_fee': 7.5, 'savings': 0.5, 'expected_avail': 2, 'et_next_out': 16.0, 'et_next_in': 21.0, 'walk_time': 2.783, 'travel_et': 17.383333333333333}, 'meterid': '630833', 'latln': (49.28193539343345, -123.12298498208641)}
IDX - 1
Time limit: 2 	 Rate: 3.0
*** Travel time: 16.7 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 2
Time limit: 2 	 Rate: 3.0
*** Travel time: 16.8 ***
out: 1 - in: 0
~~~ Parking Status: 2 ~~~
{'b_2': 1.0, 'constraint_coeffs': {'parking_fee': 4.5, 'savings': 3.5, 'expected_avail': 2, 'et_next_out': 11.0, 'et_next_in': 22.0, 'walk_time': 2.85, 'travel_et': 16.8}, 'meterid': '600802', 'latln': (49.28057177019842, -123.11893682496287)}
IDX - 3
Time limit: 2 	 Rate: 4.0
*** Travel time: 16.833333333333332 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING

{'y_1': {'b_1': 1.0,
  'constraint_coeffs': {'parking_fee': 7.5,
   'savings': 0.5,
   'expected_avail': 2,
   'et_next_out': 16.0,
   'et_next_in': 21.0,
   'walk_time': 2.783,
   'travel_et': 17.383333333333333},
  'meterid': '630833',
  'latln': (49.28193539343345, -123.12298498208641)},
 'y_2': {'b_2': 1.0,
  'constraint_coeffs': {'parking_fee': 4.5,
   'savings': 3.5,
   'expected_avail': 2,
   'et_next_out': 11.0,
   'et_next_in': 22.0,
   'walk_time': 2.85,
   'travel_et': 16.8},
  'meterid': '600802',
  'latln': (49.28057177019842, -123.11893682496287)},
 'y_3': {'b_3': 1.0,
  'constraint_coeffs': {'parking_fee': 4.5,
   'savings': 3.5,
   'expected_avail': 2,
   'et_next_out': 11.0,
   'et_next_in': 27.0,
   'walk_time': 3.9,
   'travel_et': 16.583333333333332},
  'meterid': '600827',
  'latln': (49.28009515658337, -123.1199154470978)},
 'y_4': {'b_4': 1.0,
  'constraint_coeffs': {'parking_fee': 4.5,
   'savings': 3.5,
   'expected_avail': 2,
   'et_next_out': 12.0,
   'et_nex

In [3]:
solver = Solver(variables)
solver.solve_LP()
soln = solver.get_soln_summary()
soln

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/cb5389618dcf4777956d6fe71d401753-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/cb5389618dcf4777956d6fe71d401753-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 47 RHS
At line 53 BOUNDS
At line 60 ENDATA
Problem MODEL has 5 rows, 6 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 6 (0) columns and 30 (0) elements
Perturbing problem by 0.001% of 0.79056942 - largest nonzero change 1.5849807e-05 ( 0.0031466351%) - largest zero change 0
0  Obj 0 Primal inf 3.1254395 (2)
4  Obj 1.0000202
Optimal - objective value 1
Optimal objective 1 - 4 iterations ti

,var,optimal_weight,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
1,y_2,0.896552,4.5,2,16.800000,2.850,11.0,22.0
4,y_5,0.103448,6.0,2,17.283333,3.500,15.0,26.0
5,y_6,0.000000,4.5,2,16.433333,4.350,10.0,20.0
2,y_3,0.000000,4.5,2,16.583333,3.900,11.0,27.0
3,y_4,0.000000,4.5,2,16.616667,3.667,12.0,25.0
0,y_1,0.000000,7.5,2,17.383333,2.783,16.0,21.0
